<a href="https://colab.research.google.com/github/ellozam/ciencia-datos-notebooks/blob/main/proyecto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# ✅ Instalar dependencias necesarias
!pip install gradio xlsxwriter --quiet

import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
import glob
import numpy as np
from io import BytesIO
import tempfile
import os

# ✅ Cargar el CSV del PIB
csv_path = glob.glob("/content/API_*.csv")
assert csv_path, "❌ No se encontró archivo CSV del Banco Mundial en /content"
csv_file = csv_path[0]

df = pd.read_csv(csv_file, skiprows=4)
columnas = ['Country Name'] + [str(year) for year in range(2000, 2023)]
df = df[columnas].dropna()
paises = sorted(df['Country Name'].unique().tolist())

matriz_pib = df.set_index('Country Name')
matriz_pib = matriz_pib.apply(pd.to_numeric, errors='coerce')

# ✅ Función que compara y genera archivo Excel temporal
def comparar_y_exportar(pais1, pais2):
    datos1 = matriz_pib.loc[pais1].dropna()
    datos2 = matriz_pib.loc[pais2].dropna()

    comparativa = pd.DataFrame({
        'Año': datos1.index.astype(int),
        pais1: datos1.values,
        pais2: datos2.reindex(datos1.index).values
    })
    comparativa[f"{pais1} Crec %"] = comparativa[pais1].pct_change() * 100
    comparativa[f"{pais2} Crec %"] = comparativa[pais2].pct_change() * 100
    comparativa = comparativa.round(2)

    # 🎨 Gráfico
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(comparativa['Año'], comparativa[pais1] / 1e9, label=pais1, marker='o')
    ax.plot(comparativa['Año'], comparativa[pais2] / 1e9, label=pais2, marker='o')
    ax.set_title(f'PIB: {pais1} vs {pais2}')
    ax.set_xlabel('Año')
    ax.set_ylabel('PIB (miles de millones USD)')
    ax.grid(True)
    ax.legend()
    plt.tight_layout()

    # 🖼️ Guardar gráfico en memoria
    imagen = BytesIO()
    fig.savefig(imagen, format='png')
    imagen.seek(0)

    # 📝 Crear archivo Excel temporal
    tmp_path = tempfile.NamedTemporaryFile(delete=False, suffix=".xlsx").name
    with pd.ExcelWriter(tmp_path, engine='xlsxwriter') as writer:
        comparativa.to_excel(writer, index=False, sheet_name='Comparación PIB')
        worksheet = writer.sheets['Comparación PIB']
        worksheet.insert_image('H2', 'grafico.png', {'image_data': imagen})

    # 🤖 IA recomendaciones
    def sugerir(pais_ref):
        prom = matriz_pib.loc[pais_ref].mean()
        dist = matriz_pib.apply(lambda x: np.abs(x.mean() - prom), axis=1)
        return dist.sort_values().drop(index=pais_ref).head(3).index.tolist()

    sugeridos1 = sugerir(pais1)
    sugeridos2 = sugerir(pais2)

    texto = (
        f"🔎 IA Sugerencias:\n"
        f"- Países similares a **{pais1}**: {', '.join(sugeridos1)}\n"
        f"- Países similares a **{pais2}**: {', '.join(sugeridos2)}"
    )

    return fig, comparativa, texto, tmp_path  # Gradio acepta ruta de archivo

# ✅ Interfaz Gradio
with gr.Blocks() as demo:
    gr.Markdown("## 🌍 Comparador de PIB + IA + Exportación a Excel")
    with gr.Row():
        dropdown1 = gr.Dropdown(choices=paises, label="País 1", value=paises[0])
        dropdown2 = gr.Dropdown(choices=paises, label="País 2", value=paises[1])
    boton = gr.Button("Comparar")
    grafico = gr.Plot()
    tabla = gr.Dataframe()
    texto = gr.Markdown()
    archivo = gr.File(label="Descargar archivo Excel")

    boton.click(fn=comparar_y_exportar,
                inputs=[dropdown1, dropdown2],
                outputs=[grafico, tabla, texto, archivo])

# 🚀 Lanzar app
demo.launch(share=True)




Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8432584ebb240e470f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
